## Bachelorarbeit

### Normal imports

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import seaborn as sns
import concurrent.futures
import matplotlib.pyplot as plt
from IPython.display import display
from transformers import AutoModelForSequenceClassification

### Run nb_praxisprojekt to use functions

In [ ]:
# Since we only need the functions, the execution at the bottom of the practical project notebook is not necessary and can be commented out.

%run nb_practicalproject.ipynb

### Get Stocks from NYSE Composite

In [ ]:
def extract_from_index(index):
    json_fund = leeway("FUNDAMENTALS", index)
    df_fund = pd.DataFrame.from_dict(json_fund["Components"], orient="index")[["Code", "Exchange"]]
    df_fund['ticker'] = df_fund['Code'] + '.' + df_fund['Exchange']
    df_fund.drop(['Code', 'Exchange'], axis=1, inplace=True)
    df_fund["index"] = json_fund["General"]["Code"]

    return df_fund

### Retrieve key figures and calculate avg

In [ ]:
def get_netprofitmargin(json_data, start_date, end_date):
    try:
        total_net_income = 0.0; total_revenue = 0.0;
        desired_columns = ["date", "netIncome", "totalRevenue"]
        data = json_data["Financials"]["Income_Statement"]["quarterly"]

        # Filter out items that are not in the desired date range and only keep desired columns
        filtered_json = [
            {key: item[key] for key in desired_columns if key in item} 
            for item in data if start_date <= item["date"] <= end_date
        ]

        # Filter out None values
        filtered_json = [item for item in filtered_json if all(item.values())]

        # Calculate total net income and total revenue
        for item in filtered_json:
            total_net_income += float(item["netIncome"])
            total_revenue += float(item["totalRevenue"])

        # Prevent division by zero
        if total_revenue == 0:
            return 0

        return total_net_income / total_revenue
    except Exception as e:
        print(e, data) # TODO Find reason behind: "String indices must be integers, not 'str'"

def npm_handler(function, ticker, start_date, end_date, keyfigure, _):
    netprofitmargin = get_netprofitmargin(leeway(function, ticker), start_date, end_date)
    return pd.DataFrame({keyfigure: [netprofitmargin]})

def mc_handler(function, ticker, start_date, end_date, keyfigure, column):
    df = pd.DataFrame.from_dict(leeway(function, ticker, start_date, end_date))
    df_renamed = df.rename(columns={column: keyfigure})
    return df_renamed

def dy_handler(function, ticker, start_date, end_date, keyfigure, column):
    df = pd.DataFrame.from_dict(leeway(function, ticker, start_date, end_date))
    df_renamed = df.rename(columns={column: keyfigure})
    if df.empty: 
        return pd.DataFrame({keyfigure: [-0.05]})
    return df_renamed

def get_keyfigures_for_stock(ticker, keyfigure, start_date, end_date):
    keyfigure_details = {
        "marketcap": {"function": "MARKETCAP", "handler": mc_handler, "column": "value"},
        "dividendyield": {"function": "DIVIDENDS","handler": dy_handler, "column": "yield"},
        "netprofitmargin": {"function": "FUNDAMENTALS","handler": npm_handler, "column": None}, 
    }

    if keyfigure not in keyfigure_details:
        raise ValueError("Invalid keyfigure")

    details = keyfigure_details[keyfigure]
    function = details["function"]
    handler = details["handler"]
    column = details["column"]
    
    df_renamed = handler(function, ticker, start_date, end_date, keyfigure, column)
    df_renamed["ticker"] = ticker
    return df_renamed

def finalize_data(df, keyfigure):
    df = df[["ticker", keyfigure]]
    df = df.dropna()
    if keyfigure == "netprofitmargin":
        df = df[(df[keyfigure] <= 1) & (df[keyfigure] >= -1)]
        keyfigure_dict = df.set_index("ticker")[keyfigure].to_dict()
    else:
        keyfigures = df.groupby("ticker")[keyfigure].mean().round(4)
        keyfigure_dict = keyfigures.to_dict()

    return keyfigure_dict

def get_keyfigures_for_df(df, keyfigure, start_date, end_date):
    tickers = df["ticker"].unique()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(lambda ticker: get_keyfigures_for_stock(ticker, keyfigure, start_date, end_date), tickers))

    combined_df = pd.concat(results, ignore_index=True)
    finalized_df = finalize_data(combined_df, keyfigure)

    return finalized_df

def calculate_keyfigure(df, keyfigure, start_date, end_date):
    num_of_stocks = df.shape[0]
    name_db = f"calc_{keyfigure}_{start_date}_{end_date}_{num_of_stocks}"

    keyfigure_dict = load_from_db(name_db)
    if keyfigure_dict is None:
        keyfigure_dict = get_keyfigures_for_df(df, keyfigure, start_date, end_date)
        save_to_db(keyfigure_dict, name_db)
    
    avg_keyfigure_df = pd.DataFrame.from_dict(keyfigure_dict, orient="index", columns=[keyfigure])
    return avg_keyfigure_df

### Split the data in the categories by the key figure values

In [ ]:
def split_by_keyfigure(df, keyfigure, first_threshold, second_threshold):
    first_group = df[df[keyfigure] <= first_threshold]
    second_group = df[(df[keyfigure] > first_threshold) & (df[keyfigure] <= second_threshold)]
    third_group = df[df[keyfigure] > second_threshold]

    return first_group, second_group, third_group

def get_threshold(df, keyfigure):
    static_thresholds = {
        "dividendyield": (0.00, 0.02),
        "netprofitmargin": (0.00, 0.10),
    }

    if keyfigure == "marketcap":
        first_threshold = df[keyfigure].quantile(1 / 3)
        second_threshold = df[keyfigure].quantile(2 / 3)
    elif keyfigure in static_thresholds:
        first_threshold, second_threshold = static_thresholds[keyfigure]

    return first_threshold, second_threshold

### Plot distribution

In [ ]:
def plot_distribution_keyfigure(df, keyfigure, first_threshold, second_threshold):
    keyfigure_label = keyfigure.upper()
    
    plt.hist(df[keyfigure], bins=50, alpha=0.5, color="blue")
    plt.title(f"Distribution of {keyfigure_label} with Thresholds")
    plt.xlabel(f"{keyfigure_label}")
    plt.ylabel("Frequency")

    plt.axvline(x=first_threshold, color='red', linestyle='--', linewidth=2, label='1st Threshold')
    plt.axvline(x=second_threshold, color='green', linestyle='--', linewidth=2, label='2nd Threshold')

### Remove outliers

In [ ]:
def remove_outliers(df, keyfigure, cutoff_percentile):
    threshold = df[keyfigure].quantile(1 - cutoff_percentile/100)
    filtered_df = df[df[keyfigure] < threshold]
    return filtered_df

### Get stock values

In [ ]:
def get_values_per_ticker(ticker, start_date, end_date, session):
    name_db = f"{ticker}_{start_date}_{end_date}_values" 

    # check if the data is already in the db, if not fetch it from the api
    value_dict = load_from_db(name_db)
    if value_dict is None:
        try:
            value_json = leeway("VALUE", ticker, start_date, end_date, session=session)
            value_dict = { ticker: { entry["date"]: entry["adjusted_close"] for entry in value_json } }
        except Exception as e:
            print(f"Error fetching stock values: {e}")
            value_dict = {}

        save_to_db(value_dict, name_db)

    return value_dict

def collect_valid_data(data):
    return {k: v for item in data if item is not False for k, v in item.items()}

def get_values(df, start_date, end_date):
    session = create_session()

    # add 3 months to the end date, so we get all values including these from the future date. #TODO: magic number
    adj_end_date = (pd.to_datetime(end_date) + pd.DateOffset(months=3)).strftime("%Y-%m-%d")

    unique_ticker_values = df["ticker"].unique()

    # get the stock,etf and mc values from the api using multithreading
    with concurrent.futures.ThreadPoolExecutor() as executor:
        stock_data = list(executor.map(lambda ticker: get_values_per_ticker(ticker, start_date, adj_end_date, session), unique_ticker_values))

    stock_prices = collect_valid_data(stock_data)

    return stock_prices

### Calc adjusted change

In [ ]:
def calc_adj_change_per_ticker(row, values_js, target):
    ticker = row["ticker"]
    date = row["date"]
    target_date = row[f"date_{target}"]
    stock_change = None

    # Check if the ticker exists in the values_js dictionary
    if ticker in values_js:
        stock_values = values_js[ticker]

        # Check if both dates exist for the ticker in the dictionary
        if date in stock_values and target_date in stock_values:
            stock_original_value = stock_values[date]
            stock_future_value = stock_values[target_date]

            # Calculate the percentage change
            stock_change = ((stock_future_value - stock_original_value) / stock_original_value) * 100

    return pd.Series({f"change_{target}": stock_change})

def calc_adj_change(df, values_js, target):
    df_copy = df.copy()

    # Calculate the change for each row
    df_copy[f"change_{target}"] = df_copy.apply(lambda row: calc_adj_change_per_ticker(row, values_js, target), axis=1)

    # Calculate the average change for the whole dataframe
    avg_change = df_copy[f"change_{target}"].mean()

    # Adjust the change of each row by the average change of the group
    df_copy[f"change_{target}"] = df_copy[f"change_{target}"] - avg_change
    
    return df_copy

### Load the BERT-Model

In [ ]:
def load_model(model_name):
    output_directory = "models"
    model_directory = os.path.join(output_directory, model_name)
    finetuned_model = AutoModelForSequenceClassification.from_pretrained(model_directory)
    return finetuned_model

### Evaluate the performance

In [ ]:
def evaluate_performance(sim_df, target):
    print(f"Calculations for a target in {target}")

    # get the value of all stocks we want to buy, today and in target date
    buy_signals = sim_df[sim_df["signal"] == "BUY"]
    total_change_buy_mean, total_change_buy_median = calculate_mm_change(buy_signals, "BUY")

    # get the value of all stocks we want to sell, today and in the future
    sell_signals = sim_df[sim_df["signal"] == "SELL"]
    total_change_sell_mean, total_change_sell_median = calculate_mm_change(sell_signals, "SELL")

    # check if it was a good decision to buy/sell the stocks
    total_change_mean = total_change_buy_mean + (-total_change_sell_mean)
    total_change_median = total_change_buy_median + (-total_change_sell_median)
    print(f"Win (+) or Loss (-) | Mean: {total_change_mean:.4f}%, Median: {total_change_median:.4f}%")

    eval_df = sim_df.copy()

    eval_df["error"] = eval_df["actual change"] - eval_df[f"predicted change"]

    # rmse 
    eval_df["squared_error"] = eval_df["error"] ** 2
    rmse = np.sqrt(eval_df["squared_error"].mean())

    # calculate the standard deviation of the actual change
    std_dev = eval_df["actual change"].std()
    
    return eval_df, rmse, std_dev

### More Visualizing

In [ ]:
def plot_boxplots(df1, df2, df3, signal):
    avg1, median1 = df1['actual change'].mean(), df1['actual change'].median()
    avg2, median2 = df2['actual change'].mean(), df2['actual change'].median()
    avg3, median3 = df3['actual change'].mean(), df3['actual change'].median()
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 6), sharey=True)
    
    sns.boxplot(y="actual change", data=df1, ax=axes[0], color='skyblue')
    axes[0].set_title(f'Small: {signal}\n\nAvg: {avg1:.2f}, Median: {median1:.2f}')
    
    sns.boxplot(y="actual change", data=df2, ax=axes[1], color='salmon')
    axes[1].set_title(f'Medium: {signal}\n\nAvg: {avg2:.2f}, Median: {median2:.2f}')
    
    sns.boxplot(y="actual change", data=df3, ax=axes[2], color='lightgreen')
    axes[2].set_title(f'Large: {signal}\n\nAvg: {avg3:.2f}, Median: {median3:.2f}')
    
    plt.tight_layout()
    plt.show()

### Calculate baseline

In [ ]:
def calc_baseline(df):
    print(f"Amount of rows: {len(df)}")
    display(df.head())
    print(f"Max label: {df['label'].max()}")
    avg_change = df["label"].mean()
    return avg_change

### Drop duplicates

In [ ]:
def drop_duplicates(news_df, train_news_df):

    print(f"Number of news articles before removing duplicates: {news_df.shape[0]}")

    train_news_df = train_news_df.set_index(["content", "ticker"]).index
    filtered_news_df = news_df.set_index(["content", "ticker"])

    filtered_news_df = filtered_news_df[~filtered_news_df.index.isin(train_news_df)]
    filtered_news_df = filtered_news_df.reset_index()

    print(f"Number of news articles after removing duplicates: {filtered_news_df.shape[0]}")

    return filtered_news_df

### Execution

#### Parameters

In [ ]:
index       = "NYA.INDX"
start_date  = "2021-01-01"
end_date    = "2022-12-31"
calendar    = "NYSE"
keyfigure   = "netprofitmargin"
target      = "1W"
cutoff      = 1

#### Extract the stocks from the index

In [ ]:
nyse_stocks = extract_from_index(index)
nyse_stocks.shape[0]

#### Get all possible trading days in the given time frame and choosen calendar

In [ ]:
dates_js = get_trading_days(start_date, end_date, calendar)

#### Retrieve news articles for the nyse-stocks in the given time frame

In [ ]:
news_df = get_content(nyse_stocks, start_date, end_date)
news_df = news_df.drop(columns=["etf"]) # since we only use one index, we can drop the etf column

#### Drop duplicates

In [ ]:
# Old news from the training of the model is loaded
train_etfs = ["XLP.US", "XLK.US"]
train_stocks_df = extract_from_etf(train_etfs)
train_news_df = get_content(train_stocks_df, start_date, end_date)

news_df = drop_duplicates(news_df, train_news_df)

#### Update the news-release dates to nearest valid trading day before

In [ ]:
up_news_df = update_dates(news_df, dates_js, target)

#### Get and calculate ther average keyfigure

In [ ]:
calc_keyfigures_df = calculate_keyfigure(nyse_stocks, keyfigure, start_date, end_date)
calc_keyfigures_df.shape[0]

#### Combine the news dataframe with the calculated keyvalues based on matching stocks

In [ ]:
combined_df = pd.merge(up_news_df, calc_keyfigures_df, left_on="ticker", right_index=True)

#### Calculate thresholds, either from static values or quantiles

In [ ]:
low_thresh, high_thresh = get_threshold(calc_keyfigures_df, keyfigure)

#### Remove outliers (ONLY IN VISUALIZATION)

In [ ]:
# Remove outliers from the keyfigure (ONLY IN VISUALIZATION)
calc_keyfigures_visual = remove_outliers(calc_keyfigures_df, keyfigure, cutoff)

#### Display the distribution of the keyfigure with the thresholds

In [ ]:
plot_distribution_keyfigure(calc_keyfigures_visual, keyfigure, low_thresh, high_thresh)

#### Preprocess the data

In [ ]:
prep_df = preprocess(combined_df) # preprocess the data for the model 

#### Split the keyfigures into low, medium, and high groups based on the thresholds

In [ ]:
# The amount of data per group varies, since we split equally by the tickers. Some tickers have more data than others.
low, medium, high = split_by_keyfigure(prep_df, keyfigure, low_thresh, high_thresh)

#### Get and adjust the change based on the overall group change

In [ ]:
# Get the stock values for the groups
low_values      = get_values(low, start_date, end_date)
medium_values   = get_values(medium, start_date, end_date)
high_values     = get_values(high, start_date, end_date)

# Calculate the adjusted change for the groups
# e.g. if the group goes up by 1% and the stock goes up by 2%, in relative its just a 1% increase
low_change_df       = calc_adj_change(low, low_values, target)
medium_change_df    = calc_adj_change(medium, medium_values, target)
high_change_df      = calc_adj_change(high, high_values, target)

### Get labels and adjust column names

In [ ]:
text, label, label_date = get_label_for_target(target) # get the column names for the text, label and label_date

low_change_adj_df       = adjust(low_change_df, text, label, label_date)
medium_change_adj_df    = adjust(medium_change_df, text, label, label_date)
high_change_adj_df      = adjust(high_change_df, text, label, label_date)

#### Pass the different groups through the BERT-Model

In [ ]:
model_name          = f"DBERT_{target}_V1"
mode                = "PERCENTAGE"
signal_threshold    = 10
sample_size         = 10000 # Dangerous, sample_size has to be greater than amount of tickers, or crash
random_seed         = 43    
num_pred            = 5

model               = load_model(model_name)

##### Small Group

In [ ]:
sampled_small   = sample_rows(low_change_adj_df, sample_size, random_seed) 
baseline_small  = calc_baseline(sampled_small)
print(baseline_small)

#sim_small       = simulate_predictions(model, sampled_small, mode, signal_threshold, keyfigure)
#eval_df_s, rmse_s, std_s = evaluate_performance(sim_small, target)

##### Medium Group

In [ ]:
sampled_medium  = sample_rows(medium_change_adj_df, sample_size, random_seed) 
baseline_medium = calc_baseline(sampled_medium)
print(baseline_medium)

#sim_medium      = simulate_predictions(model, sampled_medium, mode, signal_threshold, keyfigure)
#eval_df_m, rmse_m, std_m = evaluate_performance(sim_medium, target)

##### High Group

In [ ]:
sampled_large   = sample_rows(high_change_adj_df, sample_size, random_seed) 
baseline_large  = calc_baseline(sampled_large)
print(baseline_large)

#sim_large       = simulate_predictions(model, sampled_large, mode, signal_threshold, keyfigure)
#eval_df_l, rmse_l, std_l = evaluate_performance(sim_large, target)

### Boxplot

In [ ]:
# Overall boxplots
signal = "Overall"
plot_boxplots(sim_small, sim_medium, sim_large, signal)

### Stats

In [ ]:
# RMSE and STD for the groups
rmse_std_s = ((rmse_s - std_s) / std_s)
print(f"Group S | RMSE: {rmse_s:.4f}, STD: {std_s:.4f}, (RMSE - STD) / STD: {rmse_s - std_s:.4f}")

rmse_std_m = ((rmse_m - std_m) / std_m)
print(f"Group M | RMSE: {rmse_m:.4f}, STD: {std_m:.4f}, (RMSE - STD) / STD: {rmse_m - std_m:.4f}")

rmse_std_l = ((rmse_l - std_l) / std_l)
print(f"Group L | RMSE: {rmse_l:.4f}, STD: {std_l:.4f}, (RMSE - STD) / STD: {rmse_l - std_l:.4f}")

In [ ]:
sns.heatmap(eval_df_s.corr(numeric_only=True), annot=True, fmt=".3f")

In [ ]:
sns.heatmap(eval_df_m.corr(numeric_only=True), annot=True, fmt=".3f")

In [ ]:
sns.heatmap(eval_df_l.corr(numeric_only=True), annot=True, fmt=".3f")

In [ ]:
# combine the dataframes to get the average correlation
comb_df = pd.concat([eval_df_s, eval_df_m, eval_df_l])
sns.heatmap(comb_df.corr(numeric_only=True), annot=True, fmt=".3f")